In [29]:
import numpy as np

In [83]:
def create_random(m):
    myArray = []
    myArray = np.random.randint(100, size=m) + 1
    square = np.sum(myArray**2)
    ssq = np.sqrt(square)
    myArray = myArray/ssq
    return myArray

In [102]:
def find_projection(x, hash_table_size):
    hash_table = []
    size = np.shape(x)
    hash_function = create_random(size[1])

    for m in range(hash_table_size):
        hash_table.append([])

    for i in range(100):
        random_weigths = create_random(size[1])
        for m in range(size[0]):
            inner_product = np.inner(random_weigths,x[m])
            coordinates = inner_product * random_weigths
            hash_num = int(np.inner(coordinates, hash_function))
            hash_table[hash_num].append(m)
            
    return hash_table

In [82]:
c = [[1,5,77],[2,3,7],[3,11,-6]]
np.shape(c)

(3, 3)

In [104]:
x = find_projection(c, 100)
